In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import user, password

# Extract

In [2]:
# Read the recipe file and convert to dataframe
recipes = pd.read_csv("Resources/RAW_recipes.csv")
recipes = recipes.rename(columns={"name":"recipe_name",
                                  "id":"recipe_id",
                                  "minutes":"prep_time_minutes",
                                  "submitted":"date_recipe_submitted"})
recipes = recipes[['recipe_name','recipe_id','prep_time_minutes','date_recipe_submitted']]
recipes

,recipe_name,recipe_id,prep_time_minutes,date_recipe_submitted
0,arriba baked winter squash mexican style,137739,55,2005-09-16
1,a bit different breakfast pizza,31490,30,2002-06-17
2,all in the kitchen chili,112140,130,2005-02-25
3,alouette potatoes,59389,45,2003-04-14
4,amish tomato ketchup for canning,44061,190,2002-10-25
...,...,...,...,...
231632,zydeco soup,486161,60,2012-08-29
231633,zydeco spice mix,493372,5,2013-01-09
231634,zydeco ya ya deviled eggs,308080,40,2008-06-07
231635,cookies by design cookies on a stick,298512,29,2008-04-15


In [3]:
# Read the interaction file and convert to dataframe
interactions = pd.read_csv("Resources/RAW_interactions.csv")
interactions = interactions[['recipe_id','rating']]
interactions

,recipe_id,rating
0,40893,4
1,40893,5
2,44394,4
3,85009,5
4,85009,5
...,...,...
1132362,72730,0
1132363,386618,5
1132364,78003,5
1132365,78003,4


In [4]:
# Read the student survey and convert to dataframe
students = pd.read_csv("Resources/food_coded.csv")
students = students[['GPA', 'comfort_food']]
students = students.dropna()
students

,GPA,comfort_food
0,2.4,none
1,3.654,"chocolate, chips, ice cream"
2,3.3,"frozen yogurt, pizza, fast food"
3,3.2,"Pizza, Mac and cheese, ice cream"
4,3.5,"Ice cream, chocolate, chips"
...,...,...
120,3.5,"wine. mac and cheese, pizza, ice cream"
121,3,Pizza / Wings / Cheesecake
122,3.882,"rice, potato, seaweed soup"
123,3,"Mac n Cheese, Lasagna, Pizza"


# Transform

In [5]:
# Merge the two Food.com dataframes so we can search through recipe names for ratings later
recipes_updated = recipes.merge(interactions, on='recipe_id')
recipes_updated = recipes_updated.drop_duplicates('recipe_id')
recipes_updated = recipes_updated.dropna()
recipes_updated = recipes_updated.reset_index()
print(recipes_updated)

          index                                   recipe_name  recipe_id  \
0             0    arriba   baked winter squash mexican style     137739   
1             3              a bit different  breakfast pizza      31490   
2             7                     all in the kitchen  chili     112140   
3             8                            alouette  potatoes      59389   
4            10            amish  tomato ketchup  for canning      44061   
...         ...                                           ...        ...   
231631  1132351                                   zydeco soup     486161   
231632  1132357                              zydeco spice mix     493372   
231633  1132358                     zydeco ya ya deviled eggs     308080   
231634  1132363        cookies by design   cookies on a stick     298512   
231635  1132364  cookies by design   sugar shortbread cookies     298509   

        prep_time_minutes date_recipe_submitted  rating  
0                      55    

In [6]:
# Cleaning the student survey dataframe which contains various symbols and typos
students['comfort_food'] = students['comfort_food'].str.replace('/', ',')
students['comfort_food'] = students['comfort_food'].str.replace('.\r',',')
students['comfort_food'] = students['comfort_food'].str.replace('.',',')
students['comfort_food'] = students['comfort_food'].str.replace('&','and')
students['comfort_food'] = students['comfort_food'].str.replace('-',' ')
students['comfort_food'] = students['comfort_food'].str.replace('(',' ')
students['comfort_food'] = students['comfort_food'].str.replace(')',' ')
students['comfort_food'] = students['comfort_food'].str.replace('macaroni and cheese','mac n cheese')
students['comfort_food'] = students['comfort_food'].str.replace('Mac in cheese','mac n cheese')
students['comfort_food'] = students['comfort_food'].str.replace('mac and cheese','mac n cheese')
students['comfort_food'] = students['comfort_food'].str.replace('ice crea ','ice cream')
students = students.astype(str)
students

,GPA,comfort_food
0,2.4,none
1,3.654,"chocolate, chips, ice cream"
2,3.3,"frozen yogurt, pizza, fast food"
3,3.2,"Pizza, Mac and cheese, ice cream"
4,3.5,"Ice cream, chocolate, chips"
...,...,...
120,3.5,"wine, mac n cheese, pizza, ice cream"
121,3,"Pizza , Wings , Cheesecake"
122,3.882,"rice, potato, seaweed soup"
123,3,"Mac n Cheese, Lasagna, Pizza"


In [7]:
# Found more dirty data to be removed
students['GPA'] = students[students['GPA'] != 'Personal']
students['GPA'] = students[students['GPA'] != 'Personal ']
students['GPA'] = students[students['GPA'] != 'Unknown']
students['GPA'] = students['GPA'].replace('3.79 bitch', '3.79')
students['GPA'] = pd.to_numeric(students['GPA'])

In [8]:
# Select column to parse through
comfort_food = students['comfort_food']

# Make empty list for individual food
food_list = []

for food in comfort_food:
    
    if food not in food_list:
        food_list.append(food)

food_list

['none',
 'chocolate, chips, ice cream',
 'frozen yogurt, pizza, fast food',
 'Pizza, Mac and cheese, ice cream',
 'Ice cream, chocolate, chips ',
 'Candy, brownies and soda,',
 'Chocolate, ice cream, french fries, pretzels',
 'Ice cream, cheeseburgers, chips,',
 'Donuts, ice cream, chips',
 'Mac and cheese, chocolate, and pasta ',
 'Pasta, grandma homemade chocolate cake anything homemade ',
 'chocolate, pasta, soup, chips, popcorn',
 'Cookies, popcorn, and chips',
 'ice cream, cake, chocolate',
 'Pizza, fruit, spaghetti, chicken and Potatoes  ',
 'Saltfish, Candy and Kit Kat ',
 'chips, cookies, ice cream',
 'Chocolate, ice cream',
 'pizza, wings, Chinese',
 'Fast food, pizza, subs',
 'chocolate, sweets, ice cream',
 'burgers, chips, cookies',
 'Chilli, soup, pot pie',
 'Soup, pasta, brownies, cake',
 'chocolate, ice cream,milkshake, cookies',
 'Chips, ice cream, microwaveable foods ',
 'Chicken fingers, pizza ',
 'cookies, hot chocolate, beef jerky',
 'Tomato soup, pizza, Fritos, Me

In [9]:
# Make the series object into a list
items_list = [items.split(',') for items in food_list]

print(items_list)

[['none'], ['chocolate', ' chips', ' ice cream'], ['frozen yogurt', ' pizza', ' fast food'], ['Pizza', ' Mac and cheese', ' ice cream'], ['Ice cream', ' chocolate', ' chips '], ['Candy', ' brownies and soda', ''], ['Chocolate', ' ice cream', ' french fries', ' pretzels'], ['Ice cream', ' cheeseburgers', ' chips', ''], ['Donuts', ' ice cream', ' chips'], ['Mac and cheese', ' chocolate', ' and pasta '], ['Pasta', ' grandma homemade chocolate cake anything homemade '], ['chocolate', ' pasta', ' soup', ' chips', ' popcorn'], ['Cookies', ' popcorn', ' and chips'], ['ice cream', ' cake', ' chocolate'], ['Pizza', ' fruit', ' spaghetti', ' chicken and Potatoes  '], ['Saltfish', ' Candy and Kit Kat '], ['chips', ' cookies', ' ice cream'], ['Chocolate', ' ice cream'], ['pizza', ' wings', ' Chinese'], ['Fast food', ' pizza', ' subs'], ['chocolate', ' sweets', ' ice cream'], ['burgers', ' chips', ' cookies'], ['Chilli', ' soup', ' pot pie'], ['Soup', ' pasta', ' brownies', ' cake'], ['chocolate', 

In [10]:
# Split up the list so we can count occurances of each food type
items_df = pd.DataFrame(items_list).stack()
items_df

0    0             none
1    0        chocolate
     1            chips
     2        ice cream
2    0    frozen yogurt
              ...      
120  2            Pizza
121  0       Chocolates
     1            pizza
     2         and Ritz
     3                 
Length: 400, dtype: object

In [11]:
# More data cleaning - make all strings lowercase and remove excess space
items_df = items_df.str.lower()
items_df = items_df.str.strip()

# Count the occurances of food types
items_df.value_counts()

ice cream     46
pizza         37
chocolate     26
chips         23
cookies       16
              ..
cheese         1
snacks         1
beef jerky     1
chipotle       1
and egg        1
Length: 159, dtype: int64

In [12]:
# Make the series into a list and select only the top 5 results
new_food_list = items_df.value_counts().index.tolist()[0:5]

new_food_list

['ice cream', 'pizza', 'chocolate', 'chips', 'cookies']

In [13]:
# Some nan values were created along the way. They are dropped
students = students.dropna()

# After dropping nan value, need to reset the index so we can interate through loop.
students = students.reset_index()

students.head(16)

,index,GPA,comfort_food
0,0,2.400,none
1,1,3.654,"chocolate, chips, ice cream"
2,2,3.300,"frozen yogurt, pizza, fast food"
3,3,3.200,"Pizza, Mac and cheese, ice cream"
4,4,3.500,"Ice cream, chocolate, chips"
5,5,2.250,"Candy, brownies and soda,"
6,6,3.800,"Chocolate, ice cream, french fries, pretzels"
7,7,3.300,"Ice cream, cheeseburgers, chips,"
8,8,3.300,"Donuts, ice cream, chips"
9,9,3.300,"Mac and cheese, chocolate, and pasta"


In [14]:
# Drop the excess index column
students = students.drop(labels='index', axis=1)
students.head()

,GPA,comfort_food
0,2.400,none
1,3.654,"chocolate, chips, ice cream"
2,3.300,"frozen yogurt, pizza, fast food"
3,3.200,"Pizza, Mac and cheese, ice cream"
4,3.500,"Ice cream, chocolate, chips"


In [15]:
# Create empty gpa list for the loop to append all gpas related to each food type
gpa_dict={"ice cream" : [], "pizza" : [], "chocolate" : [], "chips" : [], "cookies" : []}
for food in new_food_list:
    for row in range(len(students)):
        if food in students['comfort_food'][row]:
            gpa = students['GPA'][row]
            gpa_dict[food].append(gpa)
print(gpa_dict)

{'ice cream': [3.654, 3.2, 3.8, 3.3, 3.6, 3.6, 3.4, 3.87, 2.8, 3.7, 3.65, 3.89, 3.6, 3.5, 3.2, 3.5, 3.3, 3.5, 3.9, 3.5, 4.0, 3.79, 3.0, 3.0, 3.4, 3.7, 3.68, 3.5, 3.5, 3.0, 3.6, 3.3, 3.5], 'pizza': [3.3, 2.2, 3.3, 3.0, 3.5, 3.7, 3.5, 3.292, 3.35, 3.2, 3.1, 3.7, 3.7, 2.6, 3.2, 3.5, 3.8, 3.92, 3.4, 3.7, 3.9, 3.6, 3.4, 3.5, 3.9], 'chocolate': [3.654, 3.5, 3.3, 3.5, 3.904, 3.6, 3.87, 2.8, 3.2, 3.4, 3.89, 2.9, 3.2, 3.3, 2.8, 2.6, 3.7, 3.0, 3.7, 3.0, 3.7, 3.3, 3.4], 'chips': [3.654, 3.5, 3.3, 3.3, 3.904, 3.4, 3.6, 3.7, 4.0, 3.65, 3.4, 3.5, 3.2, 3.8, 3.5, 3.292, 3.35, 2.8, 3.0, 3.2, 3.68, 3.7, 3.0, 2.9, 3.3, 3.63, 3.2], 'cookies': [3.6, 3.7, 2.8, 3.2, 4.0, 2.8, 3.4, 3.6, 3.2, 2.8, 3.9, 3.5, 3.1, 3.7, 2.6]}


In [16]:
# Visually scan through the recipe_name column. Column seems clear to proceed.
recipe_list = recipes_updated['recipe_name'].tolist()
recipe_list

['arriba   baked winter squash mexican style',
 'a bit different  breakfast pizza',
 'all in the kitchen  chili',
 'alouette  potatoes',
 'amish  tomato ketchup  for canning',
 'apple a day  milk shake',
 'aww  marinated olives',
 'backyard style  barbecued ribs',
 'bananas 4 ice cream  pie',
 'beat this  banana bread',
 'berry  good sandwich spread',
 'better than sex  strawberries',
 'better then bush s  baked beans',
 'boat house  collard greens',
 'calm your nerves  tonic',
 'chicken lickin  good  pork chops',
 'chile rellenos',
 'chinese  candy',
 'chinese  chop suey',
 'cream  of cauliflower soup  vegan',
 'cream  of spinach soup',
 'cream  of spinach soup  vegan',
 'crispy crunchy  chicken',
 'deep fried dessert thingys',
 'easiest ever  hollandaise sauce',
 'emotional balance  spice mixture',
 'fool the meat eaters  chili',
 'forgotten  minestrone',
 'fried  potatoes',
 'george s at the cove  black bean soup',
 'get the sensation  brownies',
 'global gourmet  taco casserole',
 

In [17]:
# Create empty gpa list for the loop to append all Food.com ratings related to each food type
rating_dict={"ice cream" : [], "pizza" : [], "chocolate" : [], "chips" : [], "cookies" : []}

for food in new_food_list:
    for row in range(0,len(recipes_updated)):
        if food in recipes_updated['recipe_name'][row]:
            ratings = recipes_updated['rating'][row]
            rating_dict[food].append(ratings)
    
print(rating_dict)

{'ice cream': [5, 5, 4, 4, 5, 5, 5, 5, 5, 0, 5, 4, 5, 5, 5, 5, 5, 5, 3, 5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 1, 4, 0, 4, 4, 4, 4, 5, 5, 5, 4, 1, 5, 5, 5, 4, 5, 4, 5, 5, 4, 5, 5, 5, 4, 3, 4, 5, 5, 5, 5, 4, 5, 4, 5, 5, 4, 1, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 4, 5, 5, 5, 5, 0, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 4, 1, 4, 4, 5, 2, 0, 5, 5, 4, 4, 5, 5, 0, 5, 0, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 0, 4, 1, 5, 5, 4, 5, 2, 5, 5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, 5, 5, 4, 0, 5, 5, 5, 5, 5, 5, 3, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 0, 0, 5, 3, 5, 4, 0, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 4, 5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 4, 5, 4, 2, 5, 5, 5, 5, 5, 0, 5, 5, 0, 2, 5, 1, 0, 5, 5, 4, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5

In [18]:
# Find the average GPA for each food type
avggpa_Dict = {}
for k,v in gpa_dict.items():
    avggpa_Dict[k] = sum(v)/float(len(v))
    
print(avggpa_Dict)

{'ice cream': 3.4980000000000007, 'pizza': 3.410480000000001, 'chocolate': 3.3573043478260876, 'chips': 3.4244444444444446, 'cookies': 3.326666666666667}


In [19]:
# Find the average Food.com rating for recipes containing each food type
avgrating_Dict = {}
for k,v in rating_dict.items():
    avgrating_Dict[k] = sum(v)/float(len(v))
    
print(avgrating_Dict)

{'ice cream': 4.426415094339623, 'pizza': 4.488789237668161, 'chocolate': 4.42160396485695, 'chips': 4.482323232323233, 'cookies': 4.295142378559464}


In [20]:
# Convert the GPA averages into dataframe
df_avg_gpa = pd.DataFrame.from_dict(avggpa_Dict, orient='index', columns=['average_gpa'])
df_avg_gpa = df_avg_gpa.reset_index()
df_avg_gpa

,index,average_gpa
0,ice cream,3.498000
1,pizza,3.410480
2,chocolate,3.357304
3,chips,3.424444
4,cookies,3.326667


In [21]:
# Convert the Food.com ratings averages into dataframe
df_avg_rating = pd.DataFrame.from_dict(avgrating_Dict, orient='index', columns=['average_ratings'])
df_avg_rating = df_avg_rating.reset_index()
df_avg_rating

,index,average_ratings
0,ice cream,4.426415
1,pizza,4.488789
2,chocolate,4.421604
3,chips,4.482323
4,cookies,4.295142


# Load 

In [26]:
# Create connection to PostgresSQL
rds_connection_string = user + ":" + password + "@localhost:5432/food_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

[]

In [27]:
# Send the dataframes to be queried in PostgreSQL
df_avg_gpa.to_sql(name='average_gpa', con=engine, if_exists='append', index=True)
df_avg_rating.to_sql(name='average_ratings', con=engine, if_exists='append', index=True)

In [28]:
pd.read_sql_query('SELECT * FROM average_gpa', con=engine).head()

,level_0,index,average_gpa
0,0,ice cream,3.498000
1,1,pizza,3.410480
2,2,chocolate,3.357304
3,3,chips,3.424444
4,4,cookies,3.326667


In [29]:
pd.read_sql_query('SELECT * FROM average_ratings', con=engine).head()

,level_0,index,average_ratings
0,0,ice cream,4.426415
1,1,pizza,4.488789
2,2,chocolate,4.421604
3,3,chips,4.482323
4,4,cookies,4.295142
